In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
"""
Rust Model construct, nothing need to change here
"""
import pandas as pd
import numpy as np
import random as rnd
import scipy.stats as stats
import scipy.optimize as opt
import json as json
import matplotlib as mpl
from math import exp
from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize
from IPython.display import display
from IPython.core.display import HTML
rnd.seed(2)
import warnings
warnings.filterwarnings('ignore')
from scipy.optimize import minimize


def lin_cost(s, params):
   try:
       theta1_1, = params
       return s*theta1_1/1000
   except ValueError:
       raise ValueError
       print("Wrong number of parameters specified: expected 2, got {}".format(len(params)))

class DynamicLogit(object):
    def __init__(self, data, Y, X, npars=2,dim_s =175,time=117,nbus=37,MF=lin_cost):
        """
        A statistics workbench used to evaluate the cost parameters underlying 
        a bus replacement pattern by a forward-looking agent.
        
        Takes:
            * Data: a Pandas dataframe, which contains:
                -Y: the name of the column containing the dummy exogenous 
                    variable (here, the choice)
                -X: the name of the column containing the endogenous variable 
                    (here, the state of the bus)

            * p: The state-transition vector of endogenous variable.
                    For instance, p = [0, 0.6, 0.4] means that the bus will 
                    transition to the next mileage state with probability 0.6, 
                    and to the second next mileage state with probability 0.4.

            * MF: A function passed as an argument, which is the functional 
                  form for the maintenance cost. This function must accept as
                  a first argument a state s, and as a second argument a vector
                  of parameters.
                  
            * npars: The number of parameters to evalutate (i.e. the number of 
                     parameters of the maintenance cost function, plus 1 for
                     the replacement cost)
        """   
        self.time = int(time)
        self.nbus = int(nbus)     
        
        self.endog = data.loc[:, Y].values #Choice/action
        self.exog = data.loc[:, X].values  #State
        
        self.N = self.endog.shape[0]
        self.S = int(dim_s)    
        
        # Check that the stated number of parameters correspond to the
        # specifications of the maintenance cost function.       
        try:
            MF(1, [0]*(npars-1))
        except ValueError:
            raise ValueError(("The number of parameters specified does not "
                              "match the specification of the maintenance cost"
                              " function!"))
        else:
            self.MF = MF
            self.npars = npars
        
        S = self.S

        # A second (SxS) matrix which regenerates the bus' state to 0 with
        # certainty (used to compute the replacement utility)
        self.regen_mat = np.vstack((np.ones((1, S)),np.zeros((S-1, S)))).T

    def transMat(self,theta2):    
        #theta2 =  self.fitted2.x
        S = self.S
        self.p = np.array([theta2[0],theta2[1],theta2[2],1-sum(theta2)])
        # A (SxS) matrix indicating the probability of a bus transitioning
        # from a state s to a state s' (used to compute maintenance utility)
        
        self.trans_mat = np.zeros((S, S))
        for i in range(S):
            for j, _p in enumerate(self.p):
                if i + j < S-1:
                    self.trans_mat[i][i+j] = _p
                elif i + j == S-1:
                    self.trans_mat[i][S-1] = self.p[j:].sum()
                else:
                    pass

    def belief(self,theta2,path,path_action):
        """
        This function return the belief x
        Takes:
            * theta2: the true state transition probability            
            * A path of mileage and replacement\maintenance record data
        """
        length = self.time

        x = 0

        p_0 = theta2[0]
        p_1 = theta2[1]
        p_2 = theta2[2]
        p_3 = 1-p_0-p_1-p_2
        #print(length,path)
        # count_0 = 0
        # count_1 = 0
        # count_2 = 0
        # count_3 = 0

        for i in range(length-1):
            if path_action[i] ==0:
                gap = path[i+1]-path[i]
                if gap == 0:
                    #count_0 +=1
                    x = x + np.log(p_0)
                elif gap == 1:
                    #count_1 +=1
                    x = x + np.log(p_1)
                elif gap == 2:
                    #count_2 +=1
                    x = x + np.log(p_2)
                else:
                    #count_3 +=1
                    x = x + np.log(p_3)
            else:
                gap = path[i+1]
                if gap == 0:
                    #count_0 +=1
                    x = x + np.log(p_0)
                elif gap == 1:
                    #count_1 +=1
                    x = x + np.log(p_1)
                elif gap == 2:
                    #count_2 +=1
                    x = x + np.log(p_2)
                else:
                    #count_3 +=1
                    x = x + np.log(p_3)
        return x#count_0*np.log(p_0)+count_1*np.log(p_1)+\
               #count_2*np.log(p_2)+count_3*np.log(p_3)  
    
    def myopic_costs(self, params): # - reward function
        S = self.S
        """
        This function computes the myopic expected cost associated with each 
        decision for each state.
        
        Takes:
            * A vector params, to be supplied to the maintenance cost function 
              MF. The first element of the vector is the replacement cost rc.

        Returns:
            * A (Sx2) array containing the maintenance and replacement costs 
              for the S possible states of the bus
        """
        rc = params[0]          #F : action 1
        thetas = params[1:]     #c : action 0
        maint_cost = [-self.MF(s, thetas) for s in range(0, S)]
        repl_cost = [-rc for s in range(0, S)]  #action 1
        return np.vstack((maint_cost, repl_cost)).T
    
    def fl_costs(self, params, beta=0.9999):#, threshold=1e-6, suppr_output=False): #compute V^n
        """
        Compute the non-myopic expected value of the agent for each possible 
        decision and each possible state of the bus, conditional on a vector of 
        parameters and on the maintenance cost function specified at the 
        initialization of the DynamicUtility model.

        Iterates until the difference in the previously obtained expected value 
        and the new expected value is smaller than a constant.
        
        Takes:
            * A vector params for the cost function
            * A discount factor beta (optional)
            * A convergence threshold (optional)
            * A boolean argument to suppress the output (optional)

        Returns:
            * An (Sx2) array of forward-looking costs associated with each
              state and each decision.
        """
        achieved = True
        # Initialization of the contraction mapping
        k = 0
        EV = np.ones((self.S, 1))        

        self.EV_myopic = self.myopic_costs(params)
        EV_new = np.zeros((self.S, 1))
       
        # Contraction mapping Loop
        for k in range(1000):
            EV = EV_new 
            #pchoice = self.choice_prob(EV) #\pi_theta(s,a)
            Q0 = self.EV_myopic[:,0] + beta * self.trans_mat.dot(EV).reshape(-1)
            Q1 = self.EV_myopic[:,1] + beta * EV[0]
            Q = np.vstack((Q0,Q1)).T
            
            min_cost = Q.max(1).reshape(-1,1)
            cost = Q - min_cost
            util = np.exp(cost)
            EV_new =  min_cost + np.log(util.sum(1).reshape(-1,1))
            
            k += 1

        return EV_new,Q

    def choice_prob(self, cost_array):  #\pi_theta(s,a)
        """
        Returns the probability of each choice for each observed state, 
        conditional on an array of state/decision costs (generated by the 
        myopic or forward-looking cost functions)
        """
        cost = cost_array - cost_array.max(1).reshape(-1,1)
        util = np.exp(cost)
        pchoice = util/(np.sum(util, 1).reshape(-1,1))
        return pchoice
        
    def loglike(self, params):
        """
        The log-likelihood of the Dynamic model is estimated in several steps.
        1°) The current parameters are supplied to the contraction mapping 
            function
        2°) The function returns a matrix of decision probabilities for each 
            state.
        3°) This matrix is used to compute the loglikelihood of the 
            observations
        4°) The log-likelihood are then summed accross individuals, and 
            returned
        """
        utilV,utilQ = self.fl_costs(params)#, suppr_output=True) 
        pchoice = self.choice_prob(utilQ)    

        logprob = 0
        for sample_data in range(self.N):
            action = int(self.endog[sample_data])
            state = int(self.exog[sample_data])
            logprob += np.log(pchoice[state,action]) 
        return -logprob   
        
    def fit_likelihood(self, x0=None, bounds=None):
        """
        Fit the parameters to the data.
        """
        if bounds == None:
            #bounds = [(10.5,None),(1e-6,None)]
            bounds = [(1e-6, None) for i in range(self.npars)]
            
        if x0 == None:
            x0 = [10.11,1.1]#[0.1 for i in range(self.npars)]
            
        self.fitted = minimize(self.loglike, x0=x0,bounds = bounds)
               
    def loglike2(self,theta2):
        
        self.beli = 0
        for i in range(self.nbus):
            self.beli = self.beli + self.belief(theta2,self.exog[i*self.time:(i+1)*self.time],self.endog[i*self.time:(i+1)*self.time])
        #print(theta2,self.beli)
        return -self.beli

    def fit_likelihood2(self,x0=None,bounds=None):
        """
        estimate the dynamic
        """
        bounds = [(0.001, 0.998),(0.001, 0.998),(0.001, 0.998)]
        x0 = [0.1,0.1,0.1]
        cons= ({'type': 'ineq', 'fun': lambda x: 1-x[0]-x[1]-x[2]-0.001 })

        self.fitted2 = minimize(self.loglike2, x0=x0,bounds = bounds, constraints= cons)
        self.transMat(self.fitted2.x)


# lin_to_lin = DynamicLogit(lin_dataframe, 'action','state', npars=2,dim_s =175,time=117,nbus=37,MF=lin_cost)
# lin_to_lin.fit_likelihood2()
# print(lin_to_lin.fitted2)
# #lin_to_lin.transMat([0.119,0.576,0.287])
# lin_to_lin.fit_likelihood()
# print(lin_to_lin.fitted)
# #lin_to_lin.print_parameters()
# #[0.11124183, 0.54249903, 0.30904795]

In [10]:
"""
Generate rust 1987 model's data set
Before Runing, need to change:
  path:  the path where file group4_new_175.csv is
  seed:  I use seed 0,1,2,3,4,5,6,7,8,9,10,...,100
"""
import random
import pickle
import os

path = '/content/drive/My Drive/2020_FALL/Research/RL/LiteratReview/'
seed_N = 100
seed_tol = np.arange(0,seed_N,dtype=int)  #0,1,2,3,4,...,1000


dat3 = pd.read_csv(path+'group4_new_175.csv') 

time = 117
nbus = 37

def boostrap(seed,data=dat3,time=time,nbus=nbus):
  np.random.seed(seed)
  samples = np.random.randint(0,nbus,nbus)
  state_new = np.zeros(time*nbus)
  action_new = np.zeros(time*nbus)
  for i,ss in enumerate(samples):
    j = time*i
    state_new[j:j+time] = data['state'][ss*time:(ss+1)*time]
    action_new[j:j+time] = data['action'][ss*time:(ss+1)*time]
  data = {'state':state_new,'action':action_new}
  dat_new = pd.DataFrame(data=data)
  #print(samples)
  #print(dat_new)
  return dat_new


data_tol = []
for seed in seed_tol:
    data_new = boostrap(seed,data=dat3,time=time,nbus=nbus)

    print('\n\n{} seed,\n'.format(seed))

    #Record in form: seed, logSigma, theta2, ccp, [Rc,r]
    data_record = [] 
    data_record.append(seed)
    estimation = DynamicLogit(data_new, 'action','state', npars=2,dim_s =175,
                              time=time,nbus=nbus,MF=lin_cost)
    
    #theta2 and logSigma
    estimation.fit_likelihood2()
    print(estimation.fitted2)
    
    data_record.append(estimation.fitted2.fun)
    data_record.append(estimation.fitted2.x)
    
    #[Rc,r] and ccp
    estimation.fit_likelihood()
    print(estimation.fitted) #r Rc  and CCP
    
    data_record.append(estimation.fitted.fun)
    data_record.append(estimation.fitted.x)
    
    data_tol.append(data_record)

#saving data
if not os.path.exists(path+'data'):
    os.makedirs(path+'data')
with open(path+'data/seed{}_rust.txt'.format(seed_N), "wb") as fp:   #Pickling
   pickle.dump(data_tol, fp)



0 seed,

     fun: 4373.483008344616
     jac: array([-0.32220459, -0.20953369, -0.05633545])
 message: 'Optimization terminated successfully.'
    nfev: 90
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([0.12977593, 0.56453823, 0.28751217])
      fun: 182.38539704125205
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00055138, -0.00096634])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 42
      nit: 11
   status: 0
  success: True
        x: array([9.41428343, 1.14241822])


1 seed,

     fun: 4410.265673038006
     jac: array([0.53912354, 0.54333496, 0.54241943])
 message: 'Optimization terminated successfully.'
    nfev: 90
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([0.12418445, 0.55475328, 0.30032608])
      fun: 161.8420217114709
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 8.52651283e-06, -1.39266376e-04])
  message: b'CONVERGENCE: REL_REDUCTION_

In [11]:
import numpy as np

n=len(seed_tol)

loglike2_org = np.zeros(n)
theta2_org = np.zeros((n,3))
ccp_org = np.zeros(n)
Rc_org = np.zeros((n,2))
logTol_org = np.zeros(n)
rust_org = data_tol


for i in range(n):
  loglike2_org[i] = - rust_org[i][1]
  theta2_org[i] = rust_org[i][2]
  ccp_org[i] =  - rust_org[i][3]
  Rc_org[i] = rust_org[i][4]
  logTol_org[i] = loglike2_org[i] + ccp_org[i]

In [12]:
print('Rust 1987 reward:\n',logTol_org.mean(),'\n',logTol_org.std(),'\n\n',logTol_org,'\n\n')
print('Rust 1987 reward:\n',Rc_org.mean(axis=0),'\n',Rc_org.std(axis=0),'\n\n',Rc_org,'\n\n')
print('Rust 1987 ccp:\n',ccp_org.mean(),'\n',ccp_org.std(),'\n\n',ccp_org,'\n\n')
print('Rust 1987 theta2:\n',theta2_org.mean(axis=0),'\n',theta2_org.std(axis=0),'\n\n',theta2_org,'\n\n')
print('Rust 1987:\n',loglike2_org.mean(),'\n',loglike2_org.std(),'\n\n',loglike2_org,'\n\n')

Rust 1987 reward:
 -4500.781024386864 
 55.6075836507047 

 [-4555.86840539 -4572.10769475 -4594.43091702 -4491.65219956
 -4544.98185966 -4561.36945449 -4546.79857931 -4558.41117609
 -4586.86061037 -4472.89426384 -4501.61744059 -4425.69705007
 -4509.09169767 -4543.24961385 -4467.15012636 -4472.68447076
 -4452.33427859 -4506.75130987 -4528.34924143 -4564.29535295
 -4583.28521843 -4517.68742087 -4465.62340253 -4532.03949733
 -4342.7745307  -4526.65404988 -4540.17888374 -4498.77013854
 -4500.86359584 -4467.27006195 -4456.22344757 -4394.16854037
 -4505.19123613 -4573.47705381 -4491.94169152 -4588.55021908
 -4481.23940378 -4484.3324962  -4433.67096317 -4541.88719381
 -4571.54393685 -4415.23537843 -4458.75968537 -4448.29204612
 -4504.07746131 -4494.7602436  -4583.7495671  -4531.99200439
 -4481.26429162 -4526.49150259 -4541.8481369  -4523.37251361
 -4430.37923119 -4556.14092765 -4562.90523453 -4455.63687378
 -4453.20641772 -4464.99049913 -4412.4092596  -4533.16412763
 -4505.7011393  -4324.942